# Select by Patient 

In [2]:
%%time

import IPython.display as display
import ipywidgets as widgets
import pymysql

PatientId = 1

s2conn = pymysql.connect(
    user='root',
    password='Sglstrpw34',
    host='172.31.62.112',
    port=3306,
    database='PatientRecords')

mycursor = s2conn.cursor()

sql_insert_blob_query =  """
SELECT 
    ih.PatientID
    ,ih.file
    ,pi.FirstName
    ,pi.LastName
    ,jpg.Image
    ,jpg.PredictionImage
from 
    ImageHeader ih 
join PatientInfo2 pi on
    pi.PatientID = ih.PatientID
join Images.JPGImages jpg on
    jpg.file = ih.file
where
    ih.PatientID = (%s)
order by 
    ih.PatientID, 
    ih.file; """

mycursor.execute(sql_insert_blob_query, PatientId)
    
record = mycursor.fetchall()
for row in record:
    print("Patient Id: ", row[0], "Patient Name: ", row[2], row[3]," Image ID: ", row[1] )
    image = widgets.Image(value=row[4], format='jpg', width=512, height=640)
    PredictionImage = widgets.Image(value=row[5], format='jpg', width=512, height=640) 
    a=[image,PredictionImage]
    wid=widgets.HBox(a)
    display.display(wid)
    
mycursor.close()
s2conn.close()

Patient Id:  1 Patient Name:  Anita TAYLOR  Image ID:  1a3e31be4931b71f5840e74d1431b84a


Patient Id:  1 Patient Name:  Anita TAYLOR  Image ID:  52f8ccffec591b51c73bf964ee316a49


CPU times: user 32.4 ms, sys: 14.4 ms, total: 46.9 ms
Wall time: 6.05 s


# Connect to SingleStore with at least 1 Nodule/Mass Predition

## Profile Data

In [3]:
%%time

# At least 1 Nodule/Mass Predition

s2conn = pymysql.connect(
    user='root',
    password='Sglstrpw34',
    host='172.31.62.112',
    port=3306,
    database='PatientRecords')

mycursor = s2conn.cursor()

sql =  """
SELECT 
    ih.PatientID
    ,ih.file
    ,ip2.ClassName 
    ,pi.FirstName
    ,pi.LastName
from 
    ImageHeader ih 
join (
    select
        file
        ,ClassID
        ,ClassName
        ,count(*) ClassCount
    from
        ImagePredictions ip
    group by
        file
        ,ClassID
        ,ClassName
) ip2 on
    ip2.file = ih.file
join PatientInfo2 pi on
    pi.PatientID = ih.PatientID
where
    ip2.ClassName = 'Nodule/Mass'
    and ip2.ClassCount > 1
order by 
    ih.PatientID, 
    ih.file; 
"""

mycursor.execute(sql)
print(mycursor.rowcount, "images selected.\n")

record = mycursor.fetchall()
for row in record:
    print("Patient Id: ", row[0], "Patient Name: ", row[3], row[4]," Image ID: ", row[1] , " Class Name: ", row[2])
    
mycursor.close()
s2conn.close()

770 images selected.

Patient Id:  16 Patient Name:  Kathleen LOPEZ  Image ID:  7a292f45c54655d0fa19dba160b07c4b  Class Name:  Nodule/Mass
Patient Id:  49 Patient Name:  Franklin THOMAS  Image ID:  e78d3dafa3d35c1ef2beaed26f884b84  Class Name:  Nodule/Mass
Patient Id:  117 Patient Name:  James ANDERSON  Image ID:  273a53879b3ba9afa3d6d1e3aea0a453  Class Name:  Nodule/Mass
Patient Id:  126 Patient Name:  Harold HERNANDEZ  Image ID:  35e38672875ff60d5a131d91b4db5a6d  Class Name:  Nodule/Mass
Patient Id:  127 Patient Name:  Jacqueline RODRIGUEZ  Image ID:  6c570356c5cdb674231769b436f31d1f  Class Name:  Nodule/Mass
Patient Id:  167 Patient Name:  Larry LEE  Image ID:  57a11dff1b33a1e011eb9ccd9a2f24cc  Class Name:  Nodule/Mass
Patient Id:  171 Patient Name:  Betty THOMPSON  Image ID:  acf43f7566b4f7b9fa28c839a48f9d61  Class Name:  Nodule/Mass
Patient Id:  210 Patient Name:  Mary GARCIA  Image ID:  2b3a09923dd40a6874d3ee54773575ed  Class Name:  Nodule/Mass
Patient Id:  212 Patient Name:  Joh

# Display Patient having more that 3 Nodule/Masses on X-Ray showing Images

In [4]:
%%time

s2conn = pymysql.connect(
    user='root',
    password='Sglstrpw34',
    host='172.31.62.112',
    port=3306,
    database='PatientRecords')

mycursor = s2conn.cursor()

sql =  """
 SELECT 
    ih.PatientID
    ,ih.file
    ,ip2.ClassName 
    ,pi.FirstName
    ,pi.LastName
    ,jpg.Image
    ,jpg.PredictionImage
from (
    select
        file
        ,ClassID
        ,ClassName
        ,count(*) ClassCount
    from
        ImagePredictions ip
    where
        ip.ClassName = 'Nodule/Mass'
    group by
        file
        ,ClassID
        ,ClassName
    having count(*) > 3
) ip2
join ImageHeader ih on
    ip2.file = ih.file
join PatientInfo2 pi on
    pi.PatientID = ih.PatientID
join Images.JPGImages jpg on
    jpg.file = ih.file

order by 
    ih.PatientID, 
    ih.file; 
"""

mycursor.execute(sql)
print(mycursor.rowcount, "images selected.\n")

record = mycursor.fetchall()
for row in record:
    print("Patient Id: ", row[0], "Patient Name: ", row[3], row[4]," Image ID: ", row[1] , " Class Name: ", row[2])
    image = widgets.Image(value=row[5], format='jpg', width=512, height=640)
    PredictionImage = widgets.Image(value=row[6], format='jpg', width=512, height=640) 
    a=[image,PredictionImage]
    wid=widgets.HBox(a)
    display.display(wid)
    
mycursor.close()
s2conn.close()

49 images selected.

Patient Id:  126 Patient Name:  Harold HERNANDEZ  Image ID:  35e38672875ff60d5a131d91b4db5a6d  Class Name:  Nodule/Mass


Patient Id:  296 Patient Name:  Robert MOORE  Image ID:  db724caf57a7adc90648a092e9fa2395  Class Name:  Nodule/Mass


Patient Id:  705 Patient Name:  Joy JACKSON  Image ID:  1e1dcf1ea1d974a5fea81b7616a11723  Class Name:  Nodule/Mass


Patient Id:  930 Patient Name:  Houston PEREZ  Image ID:  de20b8c80c04bd2dbe0782fb21738588  Class Name:  Nodule/Mass


Patient Id:  939 Patient Name:  Rosa MARTIN  Image ID:  4e0856fe29f4d2eecc45dcac43c39c1e  Class Name:  Nodule/Mass


...

Patient Id:  9381 Patient Name:  Donald LOPEZ  Image ID:  59615049fbc2df4b0aaeb3aca6648421  Class Name:  Nodule/Mass


Patient Id:  9512 Patient Name:  Margie JOHNSON  Image ID:  b872e348b0ba7c7259853c8312f9daa4  Class Name:  Nodule/Mass


Patient Id:  9523 Patient Name:  Patricia WHITE  Image ID:  9c9f556eea533f066368e0f4a452bee7  Class Name:  Nodule/Mass


Patient Id:  9997 Patient Name:  Carolyn THOMAS  Image ID:  c0440c09698f89df168dc146af067fe7  Class Name:  Nodule/Mass


CPU times: user 1.16 s, sys: 81.8 ms, total: 1.25 s
Wall time: 1.34 s


# Looking for concentrated Nodule/Mass - Using prediction locations

In [6]:
%%time
# Cluster analysis

s2conn = pymysql.connect(
    user='root',
    password='Sglstrpw34',
    host='172.31.62.112',
    port=3306,
    database='PatientRecords')

mycursor = s2conn.cursor()

sql =  """
 SELECT 
    ih.PatientID
    ,ih.file
    ,ip2.ClassName 
    ,pi.FirstName
    ,pi.LastName
    ,json_array_unpack(ip2.MinVector) MinVector
    ,json_array_unpack(ip2.MaxVector) MaxVector
    ,ip2.Distance
    ,jpg.Image
    ,jpg.PredictionImage
from (
    select
        file
        ,ClassID
        ,ClassName
        ,min(MinDicomVector) MinVector
        ,max(MaxDicomVector) MaxVector
        ,EUCLIDEAN_DISTANCE(min(MinDicomVector), max(MaxDicomVector)) Distance
        ,count(*) ClassCount
    from
        ImagePredictions ip
    where
        ip.ClassName = 'Nodule/Mass'
    group by
        file
        ,ClassID
        ,ClassName
    having count(*) > 1 
        -- and Distance > 2500
        and Distance < 400
) ip2
join ImageHeader ih on
    ip2.file = ih.file
join PatientInfo2 pi on
    pi.PatientID = ih.PatientID
join Images.JPGImages jpg on
    jpg.file = ih.file
order by 
    ih.PatientID, 
    ih.file; 
"""

mycursor.execute(sql)
print(mycursor.rowcount, "images selected.\n")

record = mycursor.fetchall()
for row in record:
    print("Patient Id: ", row[0], "Patient Name: ", row[3], row[4]," Image ID: ", row[1] , " Class Name: ", row[2]," Distance: ", row[7] , " Vector Points: ", row[5], row[6])
    image = widgets.Image(value=row[8], format='jpg', width=512, height=640)
    PredictionImage = widgets.Image(value=row[9], format='jpg', width=512, height=640) 
    a=[image,PredictionImage]
    wid=widgets.HBox(a)
    display.display(wid)
    
mycursor.close()
s2conn.close()

2 images selected.

Patient Id:  9373 Patient Name:  Gladys MARTIN  Image ID:  8a67f8e2673ceb7004824904b6e72055  Class Name:  Nodule/Mass  Distance:  293.8468989116611  Vector Points:  [1505,1139] [1730,1328]


Patient Id:  9454 Patient Name:  William WILSON  Image ID:  52c85584532b1c23d447ab91ac1787ef  Class Name:  Nodule/Mass  Distance:  350.59520818174343  Vector Points:  [2224,2127] [2415,2421]


CPU times: user 53.2 ms, sys: 369 µs, total: 53.6 ms
Wall time: 120 ms


# SingleStore Studio Query Tool